# get_rank()更新
由于每日能够查询的数据限制，需要更新get_rank()函数

In [1]:
import tushare as ts 
token='829a1fbce8eb0e34f05ab19906d0e08227c6f64261a81272aa078ccd'
pro=ts.pro_api(token)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import math
# 设置显示列长和行数
pd.set_option('display.max_columns', 20, 'display.min_rows', 50)
#正确显示中文和负号
plt.rcParams['font.sans-serif']=['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style({'font.sans-serif':['Arial Unicode MS', 'Arial']}) # use sns default theme and set font for Chinese

In [2]:
# set up date and time
today = time.strftime("%Y%m%d", time.localtime())
today

'20220928'

In [3]:
sd = datetime.strptime(today, "%Y%m%d") - timedelta(10)
sd

datetime.datetime(2022, 9, 18, 0, 0)

In [4]:
def get_date(date, days):
    '''
    输入日期，返回days之前的日期
    日期格式：%Y%m%d
    '''
    return (datetime.strptime(date, "%Y%m%d") - timedelta(days)).strftime("%Y%m%d")

In [5]:
get_date(today, 42)

'20220817'

In [6]:
'''
Author: Yiqian Qian
Description: file content
Date: 2022-08-24 14:00:36
LastEditors: Yiqian Qian
LastEditTime: 2022-08-24 14:30:48
FilePath: /undefined/Users/yiqianqian/Library/Mobile Documents/com~apple~CloudDocs/Development/量化交易/quantitative/lesson2/Test.ipynb
'''
# 随便取两只股票做验证
# 股票代码
data = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code')
code_list = data['ts_code'].to_list()
# 动量指标
momentum = 29  # 29天
num_days = 5  # 计算num_days天的分数
stock_dict_list = []
for code in code_list[:20]:
    score_list = []
    df = pro.daily(ts_code=code, end_date=today, start_date=get_date(
        today, momentum + num_days + 20))  # 最新的在最上面
    name = pro.stock_basic(exchange='', list_status='L',
                           ts_code=code)['name'][0]
    # print(f'{code}: {name}')
    df = df[::-1].reset_index(drop=True)  # 让最新的在最下面
    data = df.dropna()
    # 收盘价
    y = df["log"] = np.log(df.close)
    # print(f'{len(y)} data points')
    # 分析的数据个数（天）
    x = df["num"] = np.arange(df.log.size)
    # 拟合 1 次多项式
    # y = kx + b, slope 为斜率 k，intercept 为截距 b
    # slope, intercept = np.polyfit(x, y, 1)
    # 直接连接首尾点计算斜率
    slope = [(y.iloc[-1-D] - y.iloc[-momentum-D]) /
             momentum for D in range(num_days)]  # 最新的在最上面
    print(f'slope: {slope}\n')
    intercept = [y.iloc[-momentum-D] for D in range(num_days)]
    # (e ^ slope) ^ 250 - 1
    for i in range(num_days):
        annualized_returns = math.pow(math.exp(slope[i]), 250) - 1
        if i == 0: # 如果i=0，则前n天数据为df.iloc[-n::]
            r_squared = 1 - (
                sum((y.iloc[-momentum-i::] - (slope[i] * x[-momentum-i::] + intercept[i])) ** 2) /
                ((momentum - 1) * np.var(y[i:i+momentum], ddof=1))
            )
            score = annualized_returns * np.abs(r_squared)
            # print(f'score: {score}\n')
            score_list.append(score)
        else:
            r_squared = 1 - (
                sum((y.iloc[-momentum-i:-i] - (slope[i] * x[-momentum-i:-i] + intercept[i])) ** 2) /
                ((momentum - 1) * np.var(y[-momentum-i:-i], ddof=1))
            )
            score = annualized_returns * np.abs(r_squared)
            # print(f'score: {score}\n')
            score_list.append(score)
    # print(f'score list: {score_list}\n')
    stock_dict_tmp = {name: score_list}
    stock_dict_list.append(stock_dict_tmp)


slope: [-0.0007023207870494405, -0.0003715542471709848, 0.0005087712418256053, -0.00025159805787954253, 0.00022264973960805303]

slope: [0.0014403245819412179, 0.001847914022341401, 0.0024903697822002357, 0.002260972779503001, 0.002609869918364691]

slope: [-0.005183080654717869, -0.006908401064275092, -0.004544751351221818, -0.00232059115781555, -0.002485002431859367]

slope: [-0.0025266276243560626, -0.0023654591842328368, -0.0005763269301289859, -0.00038745080230778907, 0.0]

slope: [0.0010585857547864333, 0.0006568343093342907, 0.0022043823209045776, 0.0017102393496335236, 0.0026631499288272513]

slope: [-0.0005218171499610848, -0.0020560174598324883, -0.0010294125224027894, -0.00038963983634252497, -0.0009507127117616756]

slope: [-0.0031763892726829565, -0.0033993389197395705, -0.002479744192768495, -0.002037548840345054, -0.0011591934758270702]

slope: [-0.010685528144542057, -0.010163665618999809, -0.010585444411906599, -0.00903432950203428, -0.010915687936183008]

slope: [-0.0

In [7]:
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,log,num
0,000026.SZ,20220803,9.30,9.43,9.19,9.34,9.33,0.01,0.1072,31325.95,29221.759,2.234306,0
1,000026.SZ,20220804,9.36,9.48,9.24,9.46,9.34,0.12,1.2848,27814.73,26040.405,2.247072,1
2,000026.SZ,20220805,9.48,9.60,9.41,9.58,9.46,0.12,1.2685,22571.00,21484.497,2.259678,2
3,000026.SZ,20220808,9.57,10.22,9.53,9.93,9.58,0.35,3.6534,85769.81,85478.578,2.295560,3
4,000026.SZ,20220809,9.96,9.96,9.68,9.80,9.93,-0.13,-1.3092,40966.00,40112.520,2.282382,4


In [8]:
x[-29:-1]

array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
       26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36])

In [9]:
y.iloc[-29:-1]

9     2.334084
10    2.333114
11    2.334084
12    2.322388
13    2.325325
14    2.328253
15    2.310553
16    2.317474
17    2.302585
18    2.309561
19    2.316488
20    2.295560
21    2.297573
22    2.311545
23    2.314514
24    2.320425
25    2.317474
26    2.307573
27    2.306577
28    2.319442
29    2.309561
30    2.292535
31    2.254445
32    2.231089
33    2.259678
34    2.259678
35    2.247072
36    2.230014
Name: close, dtype: float64

In [10]:
#验证slope
(2.821-2.131)/29

0.023793103448275874

In [11]:
# merge list of dictionaries into one dictionary
stock_dict = {k:v for d in stock_dict_list for k,v in d.items()}
stock_dict 

{'平安银行': [-0.406643108087681,
  0.18496960355939596,
  -0.05619481443374203,
  0.006166991370881628,
  0.20637074865596705],
 '万科A': [0.07695544525662552,
  0.34700311958827346,
  0.29584360767951695,
  0.39576141716427365,
  0.5495912074375611],
 'ST国华': [-5.1092984674718895,
  -1.2563083260370886,
  -0.10413184036337642,
  -0.026316302892002128,
  -0.19496538343264672],
 'ST星源': [-3.0717113908381113,
  -0.1523380059326745,
  -0.19188229646733163,
  0.0,
  0.13814836637370784],
 '深振业A': [0.036007771336523935,
  0.43209407956383056,
  0.2865783134672959,
  0.4641251586004112,
  0.3975182656736336],
 '全新好': [-0.7347204436275395,
  -0.04047568988593222,
  -0.030480838052551255,
  -0.09340665934883843,
  -0.19973120339677786],
 '神州高铁': [-9.30182985811682,
  -0.8980621887887587,
  -0.7293232659253325,
  -0.2939218356630402,
  -0.4733257448886324],
 '中国宝安': [-4.270223537162174,
  -0.39506169638434985,
  -0.20831668862994243,
  -0.05674421143463315,
  -0.8255177842113044],
 '美丽生态': [-1.38970

In [12]:
stock_df = pd.DataFrame.from_dict(stock_dict, orient='index')
stock_df

,0,1,2,3,4
平安银行,-0.406643,0.184970,-0.056195,0.006167,0.206371
万科A,0.076955,0.347003,0.295844,0.395761,0.549591
ST国华,-5.109298,-1.256308,-0.104132,-0.026316,-0.194965
ST星源,-3.071711,-0.152338,-0.191882,0.000000,0.138148
深振业A,0.036008,0.432094,0.286578,0.464125,0.397518
全新好,-0.734720,-0.040476,-0.030481,-0.093407,-0.199731
神州高铁,-9.301830,-0.898062,-0.729323,-0.293922,-0.473326
中国宝安,-4.270224,-0.395062,-0.208317,-0.056744,-0.825518
美丽生态,-1.389705,-0.013074,-0.422724,-0.037782,0.105326
深物业A,-8.953081,-0.831283,-0.471784,-0.000052,-0.011473


In [13]:
stock_df['code'] = code_list[:20]
stock_df

,0,1,2,3,4,code
平安银行,-0.406643,0.184970,-0.056195,0.006167,0.206371,000001.SZ
万科A,0.076955,0.347003,0.295844,0.395761,0.549591,000002.SZ
ST国华,-5.109298,-1.256308,-0.104132,-0.026316,-0.194965,000004.SZ
ST星源,-3.071711,-0.152338,-0.191882,0.000000,0.138148,000005.SZ
深振业A,0.036008,0.432094,0.286578,0.464125,0.397518,000006.SZ
全新好,-0.734720,-0.040476,-0.030481,-0.093407,-0.199731,000007.SZ
神州高铁,-9.301830,-0.898062,-0.729323,-0.293922,-0.473326,000008.SZ
中国宝安,-4.270224,-0.395062,-0.208317,-0.056744,-0.825518,000009.SZ
美丽生态,-1.389705,-0.013074,-0.422724,-0.037782,0.105326,000010.SZ
深物业A,-8.953081,-0.831283,-0.471784,-0.000052,-0.011473,000011.SZ


In [14]:
name = pro.stock_basic(exchange='', list_status='L',
                           ts_code='000026.SZ')['name'][0]

name

'飞亚达'

In [15]:
# sort by latest scores
stock_df = stock_df.sort_values(by=[0], ascending=False)
stock_df

,0,1,2,3,4,code
南玻A,0.673210,0.870683,0.727609,0.932928,1.867415,000012.SZ
沙河股份,0.105544,0.398436,1.606871,2.182403,2.490278,000014.SZ
万科A,0.076955,0.347003,0.295844,0.395761,0.549591,000002.SZ
深华发A,0.051245,0.129922,0.156304,0.160238,0.019496,000020.SZ
深振业A,0.036008,0.432094,0.286578,0.464125,0.397518,000006.SZ
特力A,-0.037152,-0.159549,-0.070006,-0.477298,-0.678815,000025.SZ
深中华A,-0.067009,-0.057799,-0.092181,-0.495801,-0.391464,000017.SZ
平安银行,-0.406643,0.184970,-0.056195,0.006167,0.206371,000001.SZ
全新好,-0.734720,-0.040476,-0.030481,-0.093407,-0.199731,000007.SZ
美丽生态,-1.389705,-0.013074,-0.422724,-0.037782,0.105326,000010.SZ


In [16]:
name_list = stock_df.index.to_list()
name_list

['南玻A',
 '沙河股份',
 '万科A',
 '深华发A',
 '深振业A',
 '特力A',
 '深中华A',
 '平安银行',
 '全新好',
 '美丽生态',
 '深科技',
 'ST星源',
 '飞亚达',
 '深天地A',
 '中国宝安',
 'ST国华',
 '深康佳A',
 '深物业A',
 '神州高铁',
 '深粮控股']

In [17]:
stock_df.drop(['code'], axis=1, inplace=True)

In [21]:
stock_diff = stock_df.diff(axis=1, periods=-1).dropna(axis=1)
stock_diff[0]['神州高铁']

-8.403767669328062

In [22]:
stock_diff.loc['神州高铁']

0   -8.403768
1   -0.168739
2   -0.435401
3    0.179404
Name: 神州高铁, dtype: float64

In [23]:
np.sum(stock_diff.loc['神州高铁'] < 0)

3

In [24]:
stock_df.shift(-1, axis=1)

,0,1,2,3,4
南玻A,0.870683,0.727609,0.932928,1.867415,NaN
沙河股份,0.398436,1.606871,2.182403,2.490278,NaN
万科A,0.347003,0.295844,0.395761,0.549591,NaN
深华发A,0.129922,0.156304,0.160238,0.019496,NaN
深振业A,0.432094,0.286578,0.464125,0.397518,NaN
特力A,-0.159549,-0.070006,-0.477298,-0.678815,NaN
深中华A,-0.057799,-0.092181,-0.495801,-0.391464,NaN
平安银行,0.184970,-0.056195,0.006167,0.206371,NaN
全新好,-0.040476,-0.030481,-0.093407,-0.199731,NaN
美丽生态,-0.013074,-0.422724,-0.037782,0.105326,NaN


In [25]:
stock_var_df = (stock_df.div(stock_df.shift(-1,axis=1)) - 1).dropna(axis=1)
stock_var_df

,0,1,2,3
南玻A,-0.226803,0.196636,-0.220080,-0.500417
沙河股份,-0.735105,-0.752042,-0.263715,-0.123631
万科A,-0.778228,0.172928,-0.252470,-0.279899
深华发A,-0.605573,-0.168789,-0.024548,7.219036
深振业A,-0.916667,0.507770,-0.382541,0.167557
特力A,-0.767147,1.279086,-0.853329,-0.296865
深中华A,0.159361,-0.372989,-0.814077,0.266532
平安银行,-3.198432,-4.291578,-10.112193,-0.970117
全新好,17.152141,0.327906,-0.673676,-0.532338
美丽生态,105.298959,-0.969073,10.188409,-1.358718


In [26]:
stock_df['rising scores'] = stock_df[0] - stock_df[1]
stock_df

,0,1,2,3,4,rising scores
南玻A,0.673210,0.870683,0.727609,0.932928,1.867415,-0.197473
沙河股份,0.105544,0.398436,1.606871,2.182403,2.490278,-0.292892
万科A,0.076955,0.347003,0.295844,0.395761,0.549591,-0.270048
深华发A,0.051245,0.129922,0.156304,0.160238,0.019496,-0.078677
深振业A,0.036008,0.432094,0.286578,0.464125,0.397518,-0.396086
特力A,-0.037152,-0.159549,-0.070006,-0.477298,-0.678815,0.122397
深中华A,-0.067009,-0.057799,-0.092181,-0.495801,-0.391464,-0.009211
平安银行,-0.406643,0.184970,-0.056195,0.006167,0.206371,-0.591613
全新好,-0.734720,-0.040476,-0.030481,-0.093407,-0.199731,-0.694245
美丽生态,-1.389705,-0.013074,-0.422724,-0.037782,0.105326,-1.376631


In [28]:
stock_df.loc['神州高铁']['rising scores']

-8.403767669328062

# 股票池筛选验证

In [1]:
import jqdatasdk as jq
jq.auth('18687751766','qyqisNo.1')

auth success 


In [30]:
def get_stock_pool():
    concept_names = list(set([
        "虚拟现实",
        "元宇宙",
        "锂电池",
        "集成电路",
        "国产软件",
        "MiniLED",
        "智能穿戴",
        "智能电网",
        "智能医疗",
        "风电",
        "核电",
        "电力物联网",
        "电力改革",
        "量子通信",
        "互联网+",
        "光伏",
        "工业4.0",
        "特高压",
        "氟化工",
        "煤化工",
        "稀土永磁",
        "白酒",
        "煤炭",
        "钴",
        "盐湖提锂",
        "磷化工",
        "草甘膦",
        "航运",
        "第三代半导体",
        "太阳能",
        "柔性屏",
        "芯片",
        "新能源",
        "智能音箱",
        "苹果",
        "特斯拉",
        "宁德时代",
        "碳中和",
        "军工",
        "军民融合",
        "海工装备",
        "超级电容",
        "区块链",
        "边缘计算",
        "云计算",
        "数字货币",
        "人工智能",
        "汽车电子",
        "无人驾驶",
        "车联网",
        "网约车",
        "充电桩",
        "冷链物流",
        "OLED",
        "大飞机",
        "大数据",
        "燃料电池",
        "医疗器械",
        "生物疫苗",
        "生物医药",
        "辅助生殖",
        "健康中国",
        "基因测序",
        "超级真菌",
        "节能环保",
        "装配式建筑",
        "乡村振兴",
        "建筑节能",
        "文化传媒",
        "电子竞技",
        "网络游戏",
        "数据中心",
        "高端装备",
        '三胎',
        '养老',
        "稀缺资源",
        "稀土永磁",
        "新材料",
        "绿色电力"
    ]))

    all_concepts = jq.get_concepts()
    concept_codes = []

    for name in concept_names:
        # print(f'concept is:{name}')
        code = all_concepts[all_concepts["name"] == name].index[0]
        concept_codes.append(code)

    all_concept_stocks = []

    for concept in concept_codes:
        all_concept_stocks += jq.get_concept_stocks(concept)
    # 需要购买股票正式版6999/年！赶快赚钱！！！
    # q = jq.query(jq.valuation.code).filter(
    #     jq.valuation.market_cap >= 30,
    #     jq.valuation.market_cap <= 500,
    #     jq.valuation.code.in_(all_concept_stocks),
    # )
    # stock_df = jq.get_fundamentals(q)
    # stock_pool_all = [code for code in stock_df["code"]]
    stock_pool = [
        code
        for code in all_concept_stocks
        if not (code.startswith("30") or code.startswith("688"))
    ]  # filter 创业板和科创板

    # filter ST stock
    tags = ['ST', '退市', '退']
    stock_pool = [
        code
        for code in stock_pool if not any(tag in jq.get_security_info(code).display_name for tag in tags)
    ]
    return list(set(stock_pool))

In [31]:
get_stock_pool()

['600201.XSHG',
 '002887.XSHE',
 '002123.XSHE',
 '600375.XSHG',
 '603218.XSHG',
 '600682.XSHG',
 '002105.XSHE',
 '000002.XSHE',
 '002537.XSHE',
 '002011.XSHE',
 '601137.XSHG',
 '002682.XSHE',
 '002501.XSHE',
 '002290.XSHE',
 '000683.XSHE',
 '600215.XSHG',
 '600547.XSHG',
 '600152.XSHG',
 '600729.XSHG',
 '002993.XSHE',
 '600327.XSHG',
 '000488.XSHE',
 '603989.XSHG',
 '600173.XSHG',
 '002160.XSHE',
 '003031.XSHE',
 '601158.XSHG',
 '002739.XSHE',
 '002256.XSHE',
 '002976.XSHE',
 '605577.XSHG',
 '002531.XSHE',
 '600936.XSHG',
 '000919.XSHE',
 '002254.XSHE',
 '000519.XSHE',
 '002788.XSHE',
 '603159.XSHG',
 '600525.XSHG',
 '002449.XSHE',
 '000516.XSHE',
 '600351.XSHG',
 '002639.XSHE',
 '600521.XSHG',
 '600703.XSHG',
 '600990.XSHG',
 '000056.XSHE',
 '603396.XSHG',
 '601139.XSHG',
 '000652.XSHE',
 '601311.XSHG',
 '002232.XSHE',
 '603214.XSHG',
 '002077.XSHE',
 '601877.XSHG',
 '605366.XSHG',
 '600676.XSHG',
 '601880.XSHG',
 '000657.XSHE',
 '601177.XSHG',
 '000633.XSHE',
 '002908.XSHE',
 '003017

In [32]:
all_concepts = jq.get_concepts()
all_concepts

,name,start_date
SC0001,石墨烯,2016-07-31
SC0002,阿里,2016-07-31
SC0003,腾讯,2017-12-26
SC0004,百度,2018-06-11
SC0005,华为,2018-12-24
SC0006,今日头条,2020-04-16
SC0007,拼多多,2020-11-05
SC0008,安防,2016-07-31
SC0009,车联网,2016-07-31
SC0010,互联网金融,2016-08-09


In [33]:
all_concepts[all_concepts['name'] == '虚拟现实'].index[0]

'SC0024'

In [34]:
jq.get_concept_stocks('SC0024')

['000050.XSHE',
 '000063.XSHE',
 '000100.XSHE',
 '000156.XSHE',
 '000547.XSHE',
 '000665.XSHE',
 '000681.XSHE',
 '000687.XSHE',
 '000725.XSHE',
 '000810.XSHE',
 '000829.XSHE',
 '000839.XSHE',
 '002036.XSHE',
 '002045.XSHE',
 '002065.XSHE',
 '002079.XSHE',
 '002103.XSHE',
 '002174.XSHE',
 '002189.XSHE',
 '002222.XSHE',
 '002236.XSHE',
 '002238.XSHE',
 '002241.XSHE',
 '002253.XSHE',
 '002273.XSHE',
 '002280.XSHE',
 '002291.XSHE',
 '002292.XSHE',
 '002315.XSHE',
 '002375.XSHE',
 '002387.XSHE',
 '002416.XSHE',
 '002425.XSHE',
 '002431.XSHE',
 '002445.XSHE',
 '002456.XSHE',
 '002465.XSHE',
 '002467.XSHE',
 '002474.XSHE',
 '002475.XSHE',
 '002504.XSHE',
 '002517.XSHE',
 '002555.XSHE',
 '002558.XSHE',
 '002572.XSHE',
 '002602.XSHE',
 '002624.XSHE',
 '002632.XSHE',
 '002635.XSHE',
 '002656.XSHE',
 '002681.XSHE',
 '002699.XSHE',
 '002717.XSHE',
 '002751.XSHE',
 '002765.XSHE',
 '002776.XSHE',
 '002925.XSHE',
 '002937.XSHE',
 '002957.XSHE',
 '002992.XSHE',
 '300005.XSHE',
 '300027.XSHE',
 '300031

In [35]:
stock_names = ['退市海特', 'ST歌儿', '大展宏图']
tags = ['ST', '退市']
# 去除退市和ST股票
names = [n for n in stock_names if not any(t in n for t in tags)]
names 

['大展宏图']

In [36]:
df = pd.read_csv('rank_stock.csv', index_col = 0)
df.head()

,0,1,2,3,4,code
盈方微,313964.529443,150686.777221,37641.582421,82069.224852,132577.806871,000670.XSHE
同兴环保,1006.478679,1513.215644,964.990853,577.805077,347.875076,003027.XSHE
彩虹集团,688.775652,117.951785,674.162025,123.767898,0.532200,003023.XSHE
科士达,685.672891,999.217439,1190.365915,68.205628,3.940830,002518.XSHE
德宏股份,363.551276,253.796604,38.603610,14.484530,30.572395,603701.XSHG


In [37]:
rank_stock_dif = (
        df.drop(["code"], axis=1).diff(axis=1, periods=-1)
    )
rank_stock_dif.head()

,0,1,2,3,4
盈方微,163277.752222,113045.194800,-44427.642432,-50508.582019,NaN
同兴环保,-506.736965,548.224791,387.185776,229.930001,NaN
彩虹集团,570.823867,-556.210239,550.394126,123.235698,NaN
科士达,-313.544548,-191.148476,1122.160287,64.264798,NaN
德宏股份,109.754672,215.192995,24.119080,-16.087866,NaN


In [38]:
rank_stock_dif.dropna(axis=1, how='all').head()

,0,1,2,3
盈方微,163277.752222,113045.194800,-44427.642432,-50508.582019
同兴环保,-506.736965,548.224791,387.185776,229.930001
彩虹集团,570.823867,-556.210239,550.394126,123.235698
科士达,-313.544548,-191.148476,1122.160287,64.264798
德宏股份,109.754672,215.192995,24.119080,-16.087866


# Test New Idea

In [2]:
# get security code from name
def get_sec_code(stock_names):
    stock_codes = []
    stocks = jq.get_all_securities()
    for name in stock_names:
        stock_code = stocks[stocks['display_name'] == name].index[0]
        stock_codes.append(stock_code)
    return stock_codes

In [7]:
# convert datatime format from tushare to jq
today =datetime.strftime(datetime.strptime(today, "%Y%m%d"),"%Y-%m-%d")
today

'2022-09-26'

In [8]:
stock_code = get_sec_code(['东易日盛'])
stock_code

['002713.XSHE']

In [9]:
df = jq.get_price(stock_code, end_date=today, count=20, frequency='120m', fields=['close'])
df

,time,code,close
0,2022-09-09 11:30:00,002713.XSHE,6.14
1,2022-09-09 15:00:00,002713.XSHE,6.12
2,2022-09-13 11:30:00,002713.XSHE,6.12
3,2022-09-13 15:00:00,002713.XSHE,6.10
4,2022-09-14 11:30:00,002713.XSHE,6.00
5,2022-09-14 15:00:00,002713.XSHE,6.02
6,2022-09-15 11:30:00,002713.XSHE,5.94
7,2022-09-15 15:00:00,002713.XSHE,6.25
8,2022-09-16 11:30:00,002713.XSHE,6.11
9,2022-09-16 15:00:00,002713.XSHE,5.88


In [13]:
df.loc[df['close'].diff() != 0]

,time,code,close
0,2022-09-09 11:30:00,002713.XSHE,6.14
1,2022-09-09 15:00:00,002713.XSHE,6.12
3,2022-09-13 15:00:00,002713.XSHE,6.10
4,2022-09-14 11:30:00,002713.XSHE,6.00
5,2022-09-14 15:00:00,002713.XSHE,6.02
6,2022-09-15 11:30:00,002713.XSHE,5.94
7,2022-09-15 15:00:00,002713.XSHE,6.25
8,2022-09-16 11:30:00,002713.XSHE,6.11
9,2022-09-16 15:00:00,002713.XSHE,5.88
11,2022-09-19 15:00:00,002713.XSHE,5.94
